# Introduction

In this notebook, I will analyse the Seattle AirBNB Dataset from Inside AirBnb. I will follow the CRISP-DM process when analysing their data. The CRISP-DM stages are:

1. Business Understanding
1. Data Understanding
1. Data Preparation
1. Modelling
1. Evaluation
1. Deployment